In [1]:
import config
import cv2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing import image, text
from keras.utils import np_utils as u
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook as tqdm
import utility

Using TensorFlow backend.


In [2]:
BATCH_SIZE=8
K_FOLDS=5

In [3]:
df_train = pkl.load(open('./data/df_train.dump.pkl', 'rb'))
df_valid = pkl.load(open('./data/df_valid.dump.pkl', 'rb'))

In [4]:
df_train.head(2)

,imageId,labelId,taskId,object,attribute,label
0,1,6,5,dress,decoration,printed
1,2,7,6,dress,color,purple


In [5]:
df_valid.head(2)

,imageId,labelId,taskId,object,attribute,label
0,1,1,1,shoe,gender,men
1,1,2,2,shoe,age,adult


In [40]:
X_train = []
labels_train = []
missed = []
for idx, rows in tqdm(df_train.iterrows(), total=len(df_train)):
    img = cv2.imread(utility.image_location(rows['imageId'], config.RESIZED_TRAIN_DIR))
    if type(img) != np.ndarray:
        missed.append(rows['imageId'])
        continue
    h, w, c = img.shape
    app_img = np.empty(shape=(config.MAX_PIXEL, config.MAX_PIXEL, 3), dtype=np.uint8)
    app_img[:h, :w] = img
    X_train.append(app_img)
    labels_train.append(rows['object'])
print("%d missed." % len(missed))


2958 missed.


In [41]:
X_valid = []
labels_valid = []
missed = []
for idx, rows in tqdm(df_valid.iterrows(), total=len(df_valid)):
    img = cv2.imread(utility.image_location(rows['imageId'], config.RESIZED_VALID_DIR))
    if type(img) != np.ndarray:
        missed.append(rows['imageId'])
        continue
    h, w, c = img.shape
    app_img = np.empty(shape=(config.MAX_PIXEL, config.MAX_PIXEL, 3), dtype=np.uint8)
    app_img[:h, :w] = img
    X_valid.append(app_img)
    labels_valid.append(rows['object'])
print("%d missed." % len(missed))


532 missed.


In [42]:
X_train = np.array(X_train)
X_valid = np.array(X_valid)

In [45]:
label_tokenizer = text.Tokenizer()
label_tokenizer.fit_on_texts(labels_train)
y_train = label_tokenizer.texts_to_sequences(labels_train)
y_valid = label_tokenizer.texts_to_sequences(labels_valid)
y_train = np.array([_[0] for _ in y_train])
y_valid = np.array([_[0] for _ in y_valid])
cls_wgt = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
cls_wgt = np.array([0] + cls_wgt.tolist())
cls_wgt = {idx: value for idx, value in enumerate(cls_wgt)}
cls_wgt

{0: 0.0,
 1: 0.896300601750547,
 2: 0.9019749518304432,
 3: 1.0751784103026822,
 4: 1.1826671478841468}

In [46]:
label_tokenizer.word_index

{'dress': 2, 'outerwear': 1, 'pants': 3, 'shoe': 4}

In [47]:
X_train, X_valid = X_train.astype('float32')/255.0, X_valid.astype('float32')/255.0

In [48]:
print("Shape:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_valid:", X_valid.shape)
print("y_valid:", y_valid.shape)

Shape:
X_train: (52430, 32, 32, 3)
y_train: (52430,)
X_valid: (10473, 32, 32, 3)
y_valid: (10473,)


In [49]:
X = np.vstack((X_train, X_valid))
y = np.hstack((y_train, y_valid))

In [50]:
print("Shape:")
print("X:", X.shape)
print("y:", y.shape)

Shape:
X: (62903, 32, 32, 3)
y: (62903,)


In [70]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same',
             activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
    optimizer=SGD(momentum=0.5, decay=0.0001), metrics=['accuracy'])

In [71]:
early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = './models/object_model.hdf5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

In [72]:
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True)

In [73]:
for idx, (train_indices, valid_indices) in enumerate(skf.split(X, y)):
    print("Training on fold " + str(idx+1) + "/" + str(K_FOLDS) + "...")
    X_train, y_train = X[train_indices], y[train_indices]
    X_valid, y_valid = X[valid_indices], y[valid_indices]
    y_train = to_categorical(y_train)
    y_valid = to_categorical(y_valid)
    model.fit(
        X_train, y_train, 
        epochs=50, 
        validation_data=(X_valid, y_valid), 
        shuffle=True, 
        callbacks=[early_stopping, model_checkpoint],
        batch_size=16
    )
    break

Training on fold 1/5...
Train on 50321 samples, validate on 12582 samples
Epoch 1/50
50321/50321 [==============================] - 47s 942us/step - loss: 1.0099 - acc: 0.5794 - val_loss: 0.8626 - val_acc: 0.6512
Epoch 2/50
50321/50321 [==============================] - 47s 927us/step - loss: 0.7831 - acc: 0.6919 - val_loss: 0.7238 - val_acc: 0.7170
Epoch 3/50
50321/50321 [==============================] - 47s 932us/step - loss: 0.6843 - acc: 0.7336 - val_loss: 0.6402 - val_acc: 0.7567
Epoch 4/50
50321/50321 [==============================] - 47s 929us/step - loss: 0.6180 - acc: 0.7621 - val_loss: 0.6313 - val_acc: 0.7556
Epoch 5/50
50321/50321 [==============================] - 47s 927us/step - loss: 0.5604 - acc: 0.7858 - val_loss: 0.5791 - val_acc: 0.7807
Epoch 6/50
50321/50321 [==============================] - 47s 928us/step - loss: 0.5163 - acc: 0.8048 - val_loss: 0.5573 - val_acc: 0.7946
Epoch 7/50
50321/50321 [==============================] - 47s 932us/step - loss: 0.4726 - ac

In [74]:
print("Accuracy: %.2f" % (model.evaluate(X_valid, y_valid)[-1]*100))

12582/12582 [==============================] - 2s 156us/step
Accuracy: 83.92


In [75]:
label_tokenizer.word_index

{'dress': 2, 'outerwear': 1, 'pants': 3, 'shoe': 4}

In [76]:
import pickle as pkl
pkl.dump(label_tokenizer, open('./models/object_model_label_tokenizer.pkl', 'wb'))

In [77]:
def predict_class(img_path):
    img = cv2.imread(img_path)
    h, w, c = img.shape
    ip = np.empty(shape=(1, config.MAX_PIXEL, config.MAX_PIXEL, 3))
    ip[0, :h, :w] = img
    prediction = model.predict(ip)
    return prediction[:, 1:][0].tolist()

In [78]:
predict_class('./resized_test/1.jpg')

[0.0, 0.0, 0.0, 1.0]

In [79]:
all_rows = []
resized_test = os.listdir(config.RESIZED_TEST_DIR)
for _ in tqdm(resized_test, total=len(resized_test)):
    img_path = config.RESIZED_TEST_DIR + "/" + _
    all_rows.append([img_path] + predict_class(img_path))

In [80]:
import pandas as pd
df_test = pd.DataFrame(all_rows, columns = ['img_path', 'dress', 'outerwear', 'pants', 'shoe'])
pkl.dump(df_test, open('df_test.pkl', 'wb'))
df_test = pkl.load(open('df_test.pkl', 'rb'))
df_test.head()

,img_path,dress,outerwear,pants,shoe
0,./resized_test/2957.jpg,0.0,1.000000e+00,0.000000,0.000000e+00
1,./resized_test/11706.jpg,0.0,1.000000e+00,0.000000,7.084036e-15
2,./resized_test/10629.jpg,0.0,0.000000e+00,0.000000,1.000000e+00
3,./resized_test/9810.jpg,0.0,3.784113e-04,0.999622,0.000000e+00
4,./resized_test/13024.jpg,1.0,3.873155e-29,0.000000,0.000000e+00
